In [37]:
import glob
import zipfile
import json
import pandas as pd
import pickle
from pprint import pprint
import pandas as pd
from collections import defaultdict

In [2]:
data_path = "/opt/datasets/microsoft_kdd/"

In [3]:
paper_file = data_path + "Papers.txt"

paper_dict = {}
title_dict = {}

errs = []

with open(paper_file, "r") as f:
    for line in f:
        try:
            paper_elems = line.split("\t")
            pid = paper_elems[0]
            year = int(paper_elems[3])
            normalized_title = ''.join(filter(str.isalpha, paper_elems[1]))
            if year == "2017":
                paper_dict[pid] = paper_elems[1:]
                title_dict[normalized_title] = pid
        except Exception as ex:
            errs.appendd(line)

In [5]:
with open("kdd_papers.pickle", "wb") as t:
    pickle.dump(paper_dict, t)

with open("kdd_title.pickle", "wb") as t:
    pickle.dump(title_dict, t)

In [6]:
len(errs)

0

In [8]:
len(title_dict)

0

## Read dblp citation data

In [13]:
ls ~/data/mag_aminer/dblp.v12.7z

/home/singh_shruti/data/mag_aminer/dblp.v12.7z


In [9]:
import py7zr

In [14]:
# Extracts the file in curent location unless path specified
with py7zr.SevenZipFile('/home/singh_shruti/data/mag_aminer/dblp.v12.7z', mode='r') as z:
    z.extractall()

In [22]:
cp dblp.v12.json /home/singh_shruti/data/mag_aminer/

In [1]:
import json

In [2]:
with open("dblp.v12.json", "r") as f:
    dblp_cit = json.load(f)

In [3]:
type(dblp_cit)

list

In [4]:
dblp_cit[0]

{'id': 1091,
 'authors': [{'name': 'Makoto Satoh',
   'org': 'Shinshu University',
   'id': 2312688602},
  {'name': 'Ryo Muramatsu', 'org': 'Shinshu University', 'id': 2482909946},
  {'name': 'Mizue Kayama', 'org': 'Shinshu University', 'id': 2128134587},
  {'name': 'Kazunori Itoh', 'org': 'Shinshu University', 'id': 2101782692},
  {'name': 'Masami Hashimoto', 'org': 'Shinshu University', 'id': 2114054191},
  {'name': 'Makoto Otani', 'org': 'Shinshu University', 'id': 1989208940},
  {'name': 'Michio Shimizu',
   'org': 'Nagano Prefectural College',
   'id': 2134989941},
  {'name': 'Masahiko Sugimoto',
   'org': 'Takushoku University, Hokkaido Junior College',
   'id': 2307479915}],
 'title': 'Preliminary Design of a Network Protocol Learning Tool Based on the Comprehension of High School Students: Design by an Empirical Study Using a Simple Mind Map',
 'year': 2013,
 'n_citation': 1,
 'page_start': '89',
 'page_end': '93',
 'doc_type': 'Conference',
 'publisher': 'Springer, Berlin, Hei

In [6]:
ls

dblp.v12.json                   mag_authors_pubs_cits_2017_iclr.pickle
Filter2016data.ipynb            mag_features_2017_iclr.pickle
kdd_papers.pickle               mag_papers.pickle
kdd_title.pickle                read_mag_data.ipynb
mag_augmented_2017_iclr.pickle  read_ms_kdd.ipynb


In [7]:
len(dblp_cit)

4894081

In [19]:
papers_2017 = {}

for inf in dblp_cit:
    if "year" in inf and inf["year"] and inf["year"] == 2017:
        try:
            lowercased = inf["title"].lower()
            key = ''.join(filter(str.isalpha, lowercased))
            papers_2017[key] = inf
        except Exception as ex:
            print(ex)

In [20]:
len(papers_2017)

308588

In [21]:
data_raw = pd.read_pickle('../features/data_features_csv-2017.pkl')
data = data_raw[["id", "title", "label", "authors"]]
org_papers = data.to_dict('index')

In [22]:
found = 0

for k,v in org_papers.items():
    t = v["title"].lower()
    t = ''.join(filter(str.isalpha, t))
    if t in papers_2017:
        org_papers[k]["found"] = True
        org_papers[k]["dblp_data"] = papers_2017[t]
        found += 1
    else:
        org_papers[k]["found"] = False

In [24]:
len(org_papers), found

(490, 386)

In [30]:
refs = 0

for k, v in org_papers.items():
    if v["found"] and "references" in v["dblp_data"] and v["dblp_data"]["references"]:        
        refs += len(v["dblp_data"]["references"])

In [31]:
refs #for 177

3207

In [32]:
3207/177

18.11864406779661

### Try external APIs

In [33]:
len(org_papers)

490

In [34]:
list(org_papers.keys())[0:5]

['HJWHIKqgl', 'SyK00v5xx', 'Hkg4TI9xl', 'Hk1iOLcle', 'H178hw9ex']

In [35]:
org_papers['HJWHIKqgl']

{'id': 'HJWHIKqgl',
 'title': 'Generative Models and Model Criticism via Optimized Maximum Mean Discrepancy',
 'label': 'Accept',
 'authors': ['Dougal J. Sutherland',
  'Hsiao-Yu Tung',
  'Heiko Strathmann',
  'Soumyajit De',
  'Aaditya Ramdas',
  'Alex Smola',
  'Arthur Gretton'],
 'found': True,
 'dblp_data': {'id': 2562221994,
  'authors': [{'name': 'Dougal J. Sutherland', 'id': 2122024741},
   {'name': 'Hsiao-Yu Tung',
    'org': 'Carnegie - Mellon University#TAB#',
    'id': 2898197610},
   {'name': 'Heiko Strathmann', 'id': 1967795448},
   {'name': 'Soumyajit De', 'id': 2564638990},
   {'name': 'Aaditya Ramdas',
    'org': '"University of California, Berkeley"',
    'id': 2022285571},
   {'name': 'Alex Smola', 'id': 1972291593},
   {'name': 'Arthur Gretton', 'id': 2468960519}],
  'title': 'Generative Models and Model Criticism via Optimized Maximum Mean Discrepancy',
  'year': 2017,
  'n_citation': 21,
  'page_start': '',
  'page_end': '',
  'doc_type': 'Conference',
  'publisher

In [38]:
all_authors = defaultdict(int)

count_auths = 0

for k,v in org_papers.items():
    if "authors" in v:
        count_auths += len(v["authors"])
        
        for a in v["authors"]:
            all_authors[a] += 1

In [39]:
count_auths

1809

In [40]:
len(all_authors)

1425

In [42]:
import scholarly

In [52]:
x = scholarly.search_author('Marty Banks')

In [44]:
type(x)

generator

In [45]:
dir(x)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [53]:
p = [i for i in x]

In [54]:
len(p)

26

In [55]:
p[0]

In [62]:
print(p[0])

{'_filled': False,
 'affiliation': 'Professor of Vision Science, UC Berkeley',
 'citedby': 20174,
 'email': '@berkeley.edu',
 'id': 'Smr99uEAAAAJ',
 'interests': ['vision science', 'psychology', 'human factors', 'neuroscience'],
 'name': 'Martin Banks',
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=Smr99uEAAAAJ'}
